In [1]:
import dask
import dask.dataframe as ddf
import pandas as pd
import numpy as np
import ast
import collections
import math
from glob import glob
import pickle
import re

In [2]:
path = '/home/ubuntu/intermediate-result-new-select-editors/article-clustering/article-links/'

In [3]:
all_files = glob(path + '*')

In [4]:
page_wlinkcounts = {}
for i in all_files:
    with open(i, 'rb') as f:
        loaded_dict = pickle.load(f) 
        page_wlinkcounts.update(loaded_dict)

In [5]:
len(page_wlinkcounts)

6000

In [6]:
month = ['january','february','march','april','may','june','july','august','september','october','november','december']

In [7]:
def filter_links(counter):
    removed_keys = [i for i in counter if re.match(r'^([\s\d]+)$', i) or any(xs in i for xs in month) or re.match('\d{4}[-/]\d{2}[-/]\d{2}',i)]
    for i in removed_keys:
        del counter[i]
    return counter

In [8]:
for k,v in page_wlinkcounts.items():
    filter_links(v)

In [9]:
def cosine_similarity(counter1,counter2):
    c1squared= sum([a*a for a in counter1.values()])
    c2squared= sum([a*a for a in counter2.values()])
    c1c2=sum([counter1[k]*counter2[k]  for k in counter1.keys() if k in counter2])
    return c1c2/(math.sqrt(c1squared*c2squared))

In [10]:
%%time
page_wlink_similarities={
    key1:{
        k:v for k,v in {
            key2: cosine_similarity(page_wlinkcounts[key1],page_wlinkcounts[key2]) 
            for key2 in page_wlinkcounts.keys() 
            if not key1.startswith('Talk:') and not key2.startswith('Talk:') 
                 and page_wlinkcounts[key1] and page_wlinkcounts[key2]
            }.items() if 1>v>0.02
        } for key1 in page_wlinkcounts.keys()}

CPU times: user 2h 2min 21s, sys: 21.8 s, total: 2h 2min 43s
Wall time: 2h 2min 43s


In [11]:
with open('result/article-similarity.pkl', 'wb') as f:
    pickle.dump(page_wlink_similarities, f)

In [12]:
df = pd.DataFrame.from_dict(page_wlink_similarities,orient='index')

In [13]:
df = df.fillna(0)

In [14]:
df = df[df.index.to_list()]

In [15]:
df.to_csv("result/article-similarity-matrix.csv", sep="\t",encoding='utf-16')

In [16]:
df.to_parquet('result/article-similarity-matrix')